In [95]:
from pycorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP('http://localhost:9000')
text = 'The picture quality is great, but the value was bad. The poor battery life was disappointing. I hate the crappy battery life.'
output = nlp.annotate(text, properties = {
    'annotators': 'pos,depparse',
    'outputFormat': 'json'
})
#output['sentences'][2]['basicDependencies']
#output['sentences'][2]['tokens']

[{'after': ' ',
  'before': ' ',
  'characterOffsetBegin': 94,
  'characterOffsetEnd': 95,
  'index': 1,
  'originalText': 'I',
  'pos': 'PRP',
  'word': 'I'},
 {'after': ' ',
  'before': ' ',
  'characterOffsetBegin': 96,
  'characterOffsetEnd': 100,
  'index': 2,
  'originalText': 'hate',
  'pos': 'VBP',
  'word': 'hate'},
 {'after': ' ',
  'before': ' ',
  'characterOffsetBegin': 101,
  'characterOffsetEnd': 104,
  'index': 3,
  'originalText': 'the',
  'pos': 'DT',
  'word': 'the'},
 {'after': ' ',
  'before': ' ',
  'characterOffsetBegin': 105,
  'characterOffsetEnd': 111,
  'index': 4,
  'originalText': 'crappy',
  'pos': 'JJ',
  'word': 'crappy'},
 {'after': ' ',
  'before': ' ',
  'characterOffsetBegin': 112,
  'characterOffsetEnd': 119,
  'index': 5,
  'originalText': 'battery',
  'pos': 'NN',
  'word': 'battery'},
 {'after': '',
  'before': ' ',
  'characterOffsetBegin': 120,
  'characterOffsetEnd': 124,
  'index': 6,
  'originalText': 'life',
  'pos': 'NN',
  'word': 'life'}

In [106]:
from collections import defaultdict

def parse_corenlp_deps(sentence_json):
    adj_dict = defaultdict(list)
    for sentence in sentence_json:
        dep_list = sentence['basicDependencies']
        pos_list = sentence['tokens'] #Format is a list of objects with 'index' starting at 1
        phrase_dict = defaultdict()
        for dep in dep_list:
            if dep['dep'] == 'compound':
                phrase = dep['dependentGloss'] + ' ' + dep['governorGloss']
                phrase_dict[dep['governorGloss']] = phrase
        for dep in dep_list:
            if dep['dep'] == 'nsubj':
                noun = dep['dependentGloss']
                pos_idx = dep['dependent']
                if pos_list[pos_idx-1]['pos'] == 'NN':
                    if noun in phrase_dict:
                        noun = phrase_dict[noun]
                    adj_dict[noun].append(dep['governorGloss'])
            elif dep['dep'] == 'amod':
                noun = dep['governorGloss']
                pos_idx = dep['governor']
                if pos_list[pos_idx-1]['pos'] == 'NN':
                    if noun in phrase_dict:
                        noun = phrase_dict[noun]
                    adj_dict[noun].append(dep['dependentGloss'])
    return [tup for tup in adj_dict.items()]

In [128]:
import pickle
import csv

depparse_output = []
corpus = []
FILE_NAME = '../samples/earbuds_B000I68BD4_(N=1018_Stdev=1.34810039761).csv'
#FILE_NAME = '../samples/mouse_B000TG4BA0_(N=306_Stdev=1.38151831291).csv'

#TODO: Pickle this
with open(FILE_NAME, 'r') as csvfile:
    csvreader = csv.reader(csvfile, delimiter = ',')
    for row in csvreader:
        review_text = row[4]
        corpus.append(review_text)
        output = nlp.annotate(review_text, properties = {
            'annotators': 'pos,depparse',
            'outputFormat': 'json'
        })
        depparse_output.append(output)

In [129]:
from collections import Counter

vocab = set()
df_cnt = Counter()
for output in depparse_output:  
    deps = parse_corenlp_deps(output['sentences'])
    for phrase, adjs in deps:
        if len(adjs) > 0:
            vocab.add(phrase)
            df_cnt[phrase] += 1

df_cnt.most_common()

[('sound', 246),
 ('quality', 233),
 ('pair', 116),
 ('price', 114),
 ('noise', 73),
 ('bass', 62),
 ('set', 60),
 ('cord', 56),
 ('fit', 48),
 ('product', 40),
 ('job', 34),
 ('time', 34),
 ('music', 32),
 ('volume', 31),
 ('deal', 27),
 ('use', 26),
 ('ear', 23),
 ('problem', 23),
 ('value', 23),
 ('side', 21),
 ('way', 21),
 ('thing', 20),
 ('quality sound', 17),
 ('noise reduction', 17),
 ('earbud', 16),
 ('part', 16),
 ('range', 15),
 ('ear bud', 15),
 ('wire', 15),
 ('design', 14),
 ('something', 14),
 ('difference', 14),
 ('money', 13),
 ('Sound', 13),
 ('buy', 13),
 ('reason', 12),
 ('color', 12),
 ('nothing', 12),
 ('length', 12),
 ('size', 12),
 ('purchase', 11),
 ('bud', 11),
 ('someone', 11),
 ('anything', 10),
 ('end', 10),
 ('performance', 10),
 ('headphone', 10),
 ('shipping', 10),
 ('base', 10),
 ('issue', 9),
 ('amount', 9),
 ('company', 9),
 ('comfort', 9),
 ('packaging', 9),
 ('isolation', 9),
 ('bit', 9),
 ('replacement', 9),
 ('review', 9),
 ('plastic', 9),
 ('son'

In [127]:
for i, output in enumerate(depparse_output):
    for sentence in output['sentences']:
        dep_list = sentence['basicDependencies']
        for dep in dep_list:
            if dep['dep'] == 'amod':
                noun = dep['governorGloss']
                if noun == 'time':
                    print(dep['dependentGloss'])
                    print(corpus[i])

annoying.This
I have been using a Dell Bluetooth notebook mouse for over a year. I elected to try this one because of Microsoft's reputation and because it was on sale.Pros--Comfortable for a small mouse-Responsive movement.Cons--bad battery life - my Dell lasted 2-3 months, this lasts about 2 weeks.-No forward button -my Dell has both forward and backward buttons-No visual indication it's on -the red on other mouse assures you it isn't dead. (though I realize laser's aren't visible-how about another LED???)-No confirmation of connection -other bluetooth items flash the blue light (or something) that let you know it's connected when turned on. It does glow green when you use the power switch to turn it on.-VERY SLOW to connect when dormant. When I first 'wake-up' the mouse after sitting, it can take over 20 seconds to connect. My Dell mouse seldom took over 5 seconds.-scroll wheel center button is too sensitive. I'm always clicking the center button by accident when scrolling and it's 